## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Nikolskoye,RU,59.70,30.79,43.00,93,90,6.71,overcast clouds
1,1,Puerto Ayora,EC,-0.74,-90.35,68.00,86,70,5.99,broken clouds
2,2,Agirish,RU,61.92,63.02,24.71,98,100,4.18,overcast clouds
3,3,Faanui,PF,-16.48,-151.75,80.74,68,34,7.14,scattered clouds
4,4,Rikitea,PF,-23.12,-134.97,74.71,80,100,23.55,light rain


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
minimum_temp = float(input("What is minimum temperature you would like for your trip? "))
maximum_temp = float(input("What is maximum temperature you would like for your trip? "))

What is minimum temperature you would like for your trip? 75
What is maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= maximum_temp) & 
                                       (city_data_df["Max Temp"] >= minimum_temp)]

preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Faanui,PF,-16.48,-151.75,80.74,68,34,7.14,scattered clouds
7,7,San Cristobal,VE,7.77,-72.22,82.40,65,40,10.29,scattered clouds
11,11,Kapaa,US,22.08,-159.32,84.00,78,20,9.17,few clouds
14,14,Ormara,PK,25.21,64.64,75.88,53,0,4.38,clear sky
20,20,Hilo,US,19.73,-155.09,87.80,48,20,13.87,few clouds
26,26,Hithadhoo,MV,-0.60,73.08,83.39,68,55,6.31,broken clouds
30,30,Atambua,ID,-9.11,124.89,79.74,75,98,2.55,overcast clouds
34,34,Vanimo,PG,-2.67,141.30,80.64,82,100,2.30,moderate rain
36,36,Senanga,ZM,-16.12,23.27,79.77,19,36,9.82,scattered clouds
39,39,Biak,ID,-0.91,122.88,79.93,72,100,4.25,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

clean_df.count()

City_ID                212
City                   212
Country                212
Lat                    212
Lng                    212
Max Temp               212
Humidity               212
Cloudiness             212
Wind Speed             212
Current Description    212
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Faanui,PF,80.74,scattered clouds,-16.48,-151.75,
7,San Cristobal,VE,82.40,scattered clouds,7.77,-72.22,
11,Kapaa,US,84.00,few clouds,22.08,-159.32,
14,Ormara,PK,75.88,clear sky,25.21,64.64,
20,Hilo,US,87.80,few clouds,19.73,-155.09,
26,Hithadhoo,MV,83.39,broken clouds,-0.60,73.08,
30,Atambua,ID,79.74,overcast clouds,-9.11,124.89,
34,Vanimo,PG,80.64,moderate rain,-2.67,141.30,
36,Senanga,ZM,79.77,scattered clouds,-16.12,23.27,
39,Biak,ID,79.93,overcast clouds,-0.91,122.88,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except:
        print("Hotel not found...skipping")
        

Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping


In [9]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.loc[(hotel_df["Hotel Name"] == "")]

# Code borrowed from https://stackoverflow.com/questions/13851535/delete-rows-from-a-pandas-dataframe-based-on-a-conditional-expression-involving
clean_hotel_df = hotel_df.drop(hotel_df[hotel_df["Hotel Name"] == ""].index)

clean_hotel_df.loc[(clean_hotel_df["Hotel Name"] == "")]

clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Faanui,PF,80.74,scattered clouds,-16.48,-151.75,Bora-Bora Pearl Beach Resort & Spa
7,San Cristobal,VE,82.40,scattered clouds,7.77,-72.22,Pirineos
11,Kapaa,US,84.00,few clouds,22.08,-159.32,Sheraton Kauai Resort at Coconut Beach
20,Hilo,US,87.80,few clouds,19.73,-155.09,Hilo Hawaiian Hotel
26,Hithadhoo,MV,83.39,broken clouds,-0.60,73.08,Scoop Guest House
...,...,...,...,...,...,...,...
674,Tamandare,BR,77.29,broken clouds,-8.76,-35.10,Pousada 100 Passos
678,Panji,ID,79.83,overcast clouds,-7.73,114.10,ZAKI YAMAHA SITUBONDO
682,Marsh Harbour,BS,77.14,heavy intensity rain,26.54,-77.06,Abaco Beach Resort
683,Ullal,IN,75.20,mist,12.80,74.85,AK INTERNATIONAL LODGE


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))